In [2]:
import time
import sounddevice as sd
import threading
import numpy as np 
import numpy as np
from reservoirpy.nodes import Reservoir, Identity, RLS
import pygame
import time
import os

In [3]:
MAX_THREADS = 4  # Nombre max de threads actifs
active_threads = []  # Liste des threads en cours
lock = threading.Lock()  # Pour éviter des problèmes de threads concurrents

def non_blocking_wait(delay, start_time):
    """Attend jusqu'à ce que (start_time + delay) soit atteint sans bloquer le CPU"""
    while time.perf_counter() < start_time + delay:
        time.sleep(0.001)  # Évite une boucle active
def normalize(sample):
    sample = sample.astype(np.float32)
    return sample / np.max(np.abs(sample) + 1e-6)  # évite div par zéro

def play_sound(sound):
    try:
        sd.play(sound[0], samplerate=sound[1], blocking=True)
    except Exception as e:
        print(f"Erreur pendant lecture audio : {e}")
    finally:
        with lock:
            if threading.current_thread() in active_threads:
                active_threads.remove(threading.current_thread())

In [13]:

#######################################################DRUMS####################################################

# Init pygame mixer
pygame.mixer.init(frequency=44100, size=-16, channels=2, buffer=512)

# Reservoir config
seed = 12
nb_neurons = 6

reservoir = Reservoir(units=nb_neurons, sr=2.09, lr=0.00053, seed=seed)
readout = Identity()
reservoir <<= readout
model = reservoir >> readout

first_step = np.zeros((1, nb_neurons))
sample_played = []
penalty = 1
penalty_length = 4


# Charger les samples avec pygame
sample_paths = [
    r"samples/OneShots (24bit WAV)/Kicks/RK_BT7_Kick_04.wav",
    r"samples\OneShots (24bit WAV)\Percusion\RK_DDG1_Perc_02.wav",
    r"samples\OneShots (24bit WAV)\HiHats\RK_DDG1_Hihat_01.wav",
    r"samples\OneShots (24bit WAV)\Toms\RK_BT6_Tom_11.wav",
    r"samples\OneShots (24bit WAV)\Chords\RK_BT6_Chord_14_Fm9.wav"
]

samples = []
for path in sample_paths:
    if os.path.exists(path):
        sound = pygame.mixer.Sound(path)
        samples.append(sound)
    else:
        print(f"Fichier manquant : {path}")

# Boucle de jeu
for i in range(100):
    output = model.call(first_step)
    output[0, sample_played] -= penalty
    cache = np.argmax(output[0])

    print(f"Sample {i}: {cache}")

    if cache < len(samples):
        samples[cache].play()  

    time.sleep(0.2)

    sample_played.append(cache)
    if len(sample_played) > penalty_length:
        sample_played.pop(0)

pygame.mixer.quit()


Sample 0: 0
Sample 1: 2
Sample 2: 1
Sample 3: 4
Sample 4: 5
Sample 5: 0
Sample 6: 2
Sample 7: 1
Sample 8: 4
Sample 9: 5
Sample 10: 0
Sample 11: 2
Sample 12: 1
Sample 13: 4
Sample 14: 5
Sample 15: 0
Sample 16: 2
Sample 17: 1
Sample 18: 4
Sample 19: 5
Sample 20: 0
Sample 21: 2
Sample 22: 1
Sample 23: 4
Sample 24: 5
Sample 25: 0
Sample 26: 2
Sample 27: 1
Sample 28: 4
Sample 29: 5
Sample 30: 0
Sample 31: 2
Sample 32: 1
Sample 33: 4
Sample 34: 5
Sample 35: 0
Sample 36: 2
Sample 37: 1
Sample 38: 4
Sample 39: 5
Sample 40: 0
Sample 41: 2
Sample 42: 1
Sample 43: 4
Sample 44: 5
Sample 45: 0
Sample 46: 2
Sample 47: 1
Sample 48: 4
Sample 49: 5
Sample 50: 0
Sample 51: 2
Sample 52: 1
Sample 53: 4
Sample 54: 5
Sample 55: 0
Sample 56: 2
Sample 57: 1
Sample 58: 4
Sample 59: 5
Sample 60: 0
Sample 61: 2
Sample 62: 1
Sample 63: 4
Sample 64: 5
Sample 65: 0
Sample 66: 2
Sample 67: 1
Sample 68: 4
Sample 69: 5
Sample 70: 0
Sample 71: 2
Sample 72: 1
Sample 73: 4
Sample 74: 5
Sample 75: 0
Sample 76: 2
Sample 77

In [51]:

#######################################################CHORDS####################################################
# Init pygame mixer
pygame.mixer.init(frequency=44100, size=-16, channels=2, buffer=512)

# Reservoir config
seed = 12
nb_neurons = 6

reservoir = Reservoir(units=nb_neurons, sr=0.09, lr=0.0053, seed=seed)
readout = Identity()
reservoir <<= readout
model = reservoir >> readout

first_step = np.zeros((1, nb_neurons))
sample_played = []
penalty = 1
penalty_length = 4


# Charger les samples avec pygame
sample_paths = [
    r"samples\OneShots (24bit WAV)\Chords\RK_BT7_Chord_08_Gm.wav",
    r"samples\OneShots (24bit WAV)\Chords\RK_BT7_Chord_10_Gm.wav",
    r"samples\OneShots (24bit WAV)\Chords\RK_BT7_Chord_10_Gm.wav",
    r"samples\OneShots (24bit WAV)\HiHats\RK_BTG2_Hihat_12.wav",
    r"samples\OneShots (24bit WAV)\Chords\RK_BT7_Chord_12_Gm.wav"
]

samples = []
for path in sample_paths:
    if os.path.exists(path):
        sound = pygame.mixer.Sound(path)
        samples.append(sound)
    else:
        print(f"Fichier manquant : {path}")

# Boucle de jeu
for i in range(100):
    output = model.call(first_step)
    output[0, sample_played] -= penalty
    cache = np.argmax(output[0])

    print(f"Sample {i}: {cache}")

    if cache < len(samples):
        samples[cache].play()  

    time.sleep(0.4)

    sample_played.append(cache)
    if len(sample_played) > penalty_length:
        sample_played.pop(0)

pygame.mixer.quit()


Sample 0: 0
Sample 1: 1
Sample 2: 4
Sample 3: 5
Sample 4: 2
Sample 5: 0
Sample 6: 1
Sample 7: 4
Sample 8: 5
Sample 9: 2
Sample 10: 0
Sample 11: 1
Sample 12: 4
Sample 13: 5
Sample 14: 2
Sample 15: 0
Sample 16: 1
Sample 17: 4
Sample 18: 5
Sample 19: 2
Sample 20: 0
Sample 21: 1
Sample 22: 4
Sample 23: 5
Sample 24: 2
Sample 25: 0
Sample 26: 1
Sample 27: 4
Sample 28: 5
Sample 29: 2
Sample 30: 0
Sample 31: 1
Sample 32: 4
Sample 33: 5
Sample 34: 2
Sample 35: 0
Sample 36: 1
Sample 37: 4
Sample 38: 5
Sample 39: 2
Sample 40: 0
Sample 41: 1
Sample 42: 4
Sample 43: 5
Sample 44: 2
Sample 45: 0
Sample 46: 1
Sample 47: 4
Sample 48: 5
Sample 49: 2
Sample 50: 0
Sample 51: 1


KeyboardInterrupt: 

In [10]:
#######################################################CANARY####################################################
#               Essai de forcer le reservoir à chanter la même séquence qu'un canari                            #
#################################################################################################################

import pygame
import numpy as np
import os
import time
from reservoirpy.nodes import Reservoir, RLS

# Init pygame mixer
pygame.mixer.init(frequency=44100, size=-16, channels=2, buffer=512)

# Config
seed = 12
nb_neurons = 13
list_syll = ['Z', 'A', 'B', 'S', 'Y', 'E', 'F', 'G', 'P', 'T', 'U', 'V', 'X']
target = [0, 1 ,2, 3, 4, 10, 12, 3, 4]  # Séquence à apprendre (==ZABSYUXSY)

# Réseau
reservoir_canary = Reservoir(units=nb_neurons, sr=1.09, lr=0.553, seed=seed)
readout_canary = RLS(output_dim=len(list_syll), alpha=1e-1)

# Entrée initiale
first_step = np.zeros((1, nb_neurons))

# Mémoire pour pénaliser les répétitions
sample_played = []
penalty = 30
penalty_length = 2

# Charger les samples audio
sample_paths = [
    r"repertoire\Z-79910-134902-014-rouge6_July_01_2014_53231019.wav",
    r"repertoire\A-40572-76382-016-rouge6_July_01_2014_53296189.wav",
    r"repertoire\B-242374-303894-050-rouge6_July_01_2014_42083178.wav",
    r"repertoire\S1-131463-201846-043-rouge6_July_01_2014_41919004.wav",
    r"repertoire\Y-556587-573565-036-rouge6_July_01_2014_34575943.wav", 
    r"repertoire\E1-421685-460669-035-rouge6_July_01_2014_34338140.wav", 
    r"repertoire\F1-160039-256177-046-rouge6_July_01_2014_42000356.wav", 
    r"repertoire\G-50980-56272-004-rouge6_July_01_2014_52824900.wav", 
    r"repertoire\P1-289252-347729-001-rouge6_July_01_2014_42678325.wav", 
    r"repertoire\T-96844-151704-053-rouge6_July_01_2014_42192245.wav", 
    r"repertoire\U-349096-359283-033-rouge6_July_01_2014_34297830.wav", 
    r"repertoire\V-350066-353727-021-rouge6_July_01_2014_53390980.wav", 
    r"repertoire\X-225440-289252-001-rouge6_July_01_2014_42678325.wav"

]

samples = []
for path in sample_paths:
    if os.path.exists(path):
        sound = pygame.mixer.Sound(path)
        samples.append(sound)
    else:
        print(f"Fichier manquant : {path}")

# Boucle principale
for idx in range(10):
    for i in range(9):
        state = reservoir_canary(first_step)              # État du reservoir
        output = readout_canary(state)                    # Prédiction

        if sample_played:
            output[0, sample_played] -= penalty    # Appliquer une pénalité aux choix récents

        cache = np.argmax(output[0])               # Syllabe prédite (index)
        print(f"Sample {i}: {list_syll[cache]}")

        if cache < len(samples):
            samples[cache].play()

        time.sleep(0.8)

        sample_played.append(cache)
        if len(sample_played) > penalty_length:
            sample_played.pop(0)

        # Entraînement 
        if i < len(target):
            teacher = np.zeros(len(list_syll))
            teacher[target[i]] = 1 
            readout_canary.train(state[0], teacher)

pygame.mixer.quit()


Sample 0: Z
Sample 1: A
Sample 2: B
Sample 3: S
Sample 4: Y
Sample 5: E
Sample 6: U
Sample 7: X
Sample 8: S
Sample 0: Y
Sample 1: Z
Sample 2: A
Sample 3: B
Sample 4: S
Sample 5: Y
Sample 6: U
Sample 7: X
Sample 8: S
Sample 0: Y
Sample 1: Z
Sample 2: S
Sample 3: Y
Sample 4: A
Sample 5: S
Sample 6: Y
Sample 7: X
Sample 8: S
Sample 0: Y
Sample 1: Z
Sample 2: S
Sample 3: Y
Sample 4: A
Sample 5: S
Sample 6: Y
Sample 7: X
Sample 8: S
Sample 0: Y
Sample 1: Z
Sample 2: S
Sample 3: Y
Sample 4: A
Sample 5: S
Sample 6: Y
Sample 7: X
Sample 8: S
Sample 0: Y
Sample 1: Z
Sample 2: S


KeyboardInterrupt: 

In [55]:

#######################################################NATURE####################################################
# Init pygame mixer
pygame.mixer.init(frequency=44100, size=-16, channels=2, buffer=512)

# Reservoir config
seed = 12
nb_neurons = 12

reservoir = Reservoir(units=nb_neurons, sr=1.09, lr=0.7053, seed=seed)
readout = Identity()
reservoir <<= readout
model = reservoir >> readout

first_step = np.zeros((1, nb_neurons))
sample_played = []
penalty = 1
penalty_length = 16


# Charger les samples avec pygame
sample_paths = [
    r"repertoire\S1-317035-394254-023-rouge6_July_01_2014_33461156.wav",
    r"repertoire\A2-178253-225440-007-rouge6_July_01_2014_53035723.wav",
    r"repertoire\B-242374-303894-050-rouge6_July_01_2014_42083178.wav",
    r"nature_effect\river.mp3",
    r"repertoire\E1-648667-676935-043-rouge6_July_01_2014_41919004.wav", 
    r"nature_effect\Wind Sound SOUND EFFECT - No Copyright[Download Free].wav", 
    r"repertoire\T-259926-319417-018-rouge6_July_01_2014_53296189.wav", 
    r"repertoire\U-416878-438751-028-rouge6_July_01_2014_34254613.wav", 
    r"repertoire\Y-292692-349096-029-rouge6_July_01_2014_34254613.wav", 
    r"repertoire\Z-56272-87407-004-rouge6_July_01_2014_52824900.wav", 
    r"nature_effect\owl.mp3", 
    r"nature_effect\Chaffinch Bird Sound Effect.mp3"
]

samples = []
for path in sample_paths:
    if os.path.exists(path):
        sound = pygame.mixer.Sound(path)
        samples.append(sound)
    else:
        print(f"Fichier manquant : {path}")
try : 
    # Boucle de jeu
    for i in range(100):
        output = model.call(first_step)
        output[0, sample_played] -= penalty
        cache = np.argmax(output[0])

        print(f"Sample {i}: {cache}")

        if cache < len(samples):
            samples[cache].play()  

        time.sleep(4)

        sample_played.append(cache)
        if len(sample_played) > penalty_length:
            sample_played.pop(0)
except KeyboardInterrupt:
    for sample in samples:
        sample.stop()

pygame.mixer.quit()


Sample 0: 0
Sample 1: 1
Sample 2: 2
Sample 3: 3
Sample 4: 9
Sample 5: 4
